# WaporForPixswab 

This notebook can be used to downaload WaPOR data and create netCDF files to be used in PixSWAB model.
The data for wich netCDF files are created are LCC, PCP, I and AETI and the time step is monthly. The LCC data is yearly. I availbale at dekadal frequency, so the code aggregated the dekadal to monthly values for I.

The levels of the data and the sahpe file of the ROI is provided in a config file (WaporForPixswab.ini).

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import glob
import sys
import pandas as pd
import geopandas as gpd
import datetime
import calendar

folder=r"WaPOR"
sys.path.append(folder) #add folder with local modules to system paths
import WaPOR #import local module 'WaPOR'
from configparser import ConfigParser
import ast
import numpy as np
import xarray as xr
import rioxarray as rio
import netCDF4
from tqdm.auto import tqdm
import WaPOR_create_nc as ncf
import shutil
import time


### download and create netCDFs

In [ ]:
t = time.time()
# Get WaPOR products list
WaPOR_products_lst = WaPOR.API.getCatalog().code.tolist()

# read the config file to get the varibales, thier extent and frequey and path to folder to download to
config_file_path = r'WaporForPixswab.ini'
parser = ConfigParser()
parser.read(config_file_path)

output_dir = parser.get('FILE_PATHS','PathOut')
output_dir_tif = os.path.join(output_dir, 'tif')
output_dir_nc = os.path.join(output_dir, 'nc')
# create directories

if not os.path.exists(output_dir_nc):
  os.makedirs(output_dir_nc)
# get dates
Startdate = datetime.datetime.strptime(parser.get('PERIOD','Start_date'),"%d/%m/%Y")
Enddate = datetime.datetime.strptime(parser.get('PERIOD','End_data'),"%d/%m/%Y")

years = pd.date_range(Startdate,Enddate,freq='AS').to_pydatetime()
months = pd.date_range(Startdate,Enddate,freq='MS').to_pydatetime()


# frequency sysmbols
sym_freq = {
            'dekadal':'D',
            'monthly':'M',
            'yearly': 'A'
        }

lst_variables_raster = ast.literal_eval(parser.get('RASTER_TO_DOWNLOAD', 'variablesList'))
basin_shape =parser.get('RASTER_TO_DOWNLOAD', 'basinShape')
shape = ncf.get_shp(basin_shape)

delta_box = float(parser.get('RASTER_TO_DOWNLOAD','delta_box'))
xmin,ymin,xmax,ymax = shape.geometry.total_bounds
latlim = [ymin-delta_box, ymax+delta_box]
lonlim = [xmin-delta_box, xmax+delta_box]

lst_variables_raster1 = [s + '_R' for s in lst_variables_raster]
data2dl = []
for var in lst_variables_raster1:
    var_name = var.split('_')[0]
    prod = 'L'+parser.get(var, 'level')+'_'+var_name+'_'+sym_freq[parser.get(var, 'freq').lower()]
    data2dl.append(prod)

# check if the product to download is in WaPOR products
not_WaPOR_prod = [i for i in data2dl if i not in WaPOR_products_lst]
if not_WaPOR_prod:
    print('{} is not in WaPOR products list. Correct the infor in the config file'.format(not_WaPOR_prod))

data2dl = list(set(data2dl) - set(not_WaPOR_prod))
order = [data2dl.index("L2_LCC_A"), data2dl.index("L2_I_D"),data2dl.index("L2_AETI_M"),data2dl.index("L1_PCP_M")]
data2dl = [data2dl[i] for i in order]

#download products which are in WaPOR products list
temp_file = os.path.join(output_dir, 'template.tif')
paths = []
start_time = Startdate
for prod in data2dl:
    name, frq, level = ncf.get_prod_names(prod,sym_freq)
    # print(frq)
    # print(prod,'\n *processing')
    if('LCC' in prod):
        no_times = years
    else:
        no_times = months
            
    frqsym = sym_freq[frq]
    Dir, df_avail, bbox, cube_code = WaPOR.WaporInfo(
                        output_dir_tif,
                        data=name,
                        freq = frqsym,
                        Startdate=Startdate,
                        Enddate=Enddate,
                        latlim=latlim,
                        lonlim=lonlim,
                        level=level,
                        Waitbar = 0
                    )
            
    for i in range(len(no_times)-1):
        start_date = no_times[i]
        end_date = no_times[i+1]

        if('LCC' in prod):
            tcol = pd.to_datetime(df_avail.iloc[:,0])
            df_period = df_avail.loc[(start_date <= tcol) & (tcol < end_date)]
        else:
            tcol = pd.to_datetime(df_avail.iloc[:,0].str[:7])
            df_period = df_avail.loc[(start_date <= tcol) & (tcol < end_date)]

            for index,row in df_period.iterrows(): 
            # print('   ',row['raster_id'])
            dh = WaPOR.download(Dir, bbox, cube_code, row['time_code'], row['raster_id'])
            
        # pass the arguments to teh func to downlaod           
        if(('LCC' in prod) & (i == 0)):
            template_file = glob.glob(os.path.join(dh,'*.tif'))[-1]
            shutil.copy2(template_file, temp_file)
            template = ncf.get_template(temp_file, shape)

        if('_I_' in prod):
            fh = glob.glob(os.path.join(dh,'*.tif'))
            cc = pd.Timestamp(start_date).days_in_month
            dk3 = cc - 20
            da_lst = []
            for f in fh:
                with rio.open_rasterio(f) as da:
                    da_lst.append(da.load())
                da.close()
                os.remove(f)
                
            ds = xr.concat(da_lst, dim = 'band') 

            dkds = [10,10,dk3] 
            dk_len = xr.DataArray(
                dkds,
                dims=['band'],
                coords=[list(ds.coords['band'].values)],
            )
            # multiply the depth by the dataset
            with xr.set_options(keep_attrs=True): 
                ds_I = (ds * dk_len).sum(dim = 'band')
            ds.close()
            dk_len.close()
            # ncf.delet_dir2(dh)
        key = prod.split('_')[1]
        if(i == 0):
            if (key =='LCC'):
                freq = 'yearly'
                unit = '-'
                sf = 1
            else:
                freq = 'monthly'
                unit = 'mm/month'
                sf = 0.1
            # print(key.lower(), freq)
            nc_fn=os.path.join(output_dir_nc, key.lower()+'_'+freq+ '.nc')
            dims = {'time':  None, 'latitude': template.y.values,
                    'longitude': template.x.values}

            dim = tuple(dims)
            attrs = {
                        'units': unit,
                        'quantity':key,
                        'source': 'WaPOR_v2',
                        'period':freq,
                        # 'scale_factor': sf
                    }
            var_dict = {
                    key:[dim,attrs]
                  }
            ncf.init_nc(nc_fn, dims, var_dict, fill = -9999, attr = attrs)

            chunk_x = min(len(template.x), 2000)  
            chunk_y = min(len(template.y), 2000)
            chunks = {'x':chunk_x, 'y':chunk_y }

        if('_I_' in prod):
            ds = ds_I*sf
        else:
            fh_src = glob.glob(os.path.join(dh,'*.tif'))[0]
            with rio.open_rasterio(fh_src,chunks=chunks) as da:
                ds = da.load()*sf
            da.close()
            ds = ds.squeeze(dim = 'band', drop = True)
            os.remove(fh_src)

        ds_interpolated = (
            ds.rio.reproject_match(template, resampling=1)
            .assign_coords({"x": template.x, "y": template.y})
        )
        ds.close()
        del ds

        ds_interpolated = ds_interpolated+template*0
        # ds_interpolated = ds_interpolated.drop(['spatial_ref'])

        var = dict()        
        var[key] = ds_interpolated
        ncf.fill_nc_one_timestep(nc_fn, var, np.datetime64(start_date.date()))

    ncf.delet_dir2(dh)
        
elapsed = time.time() - t
print(">> Time elapsed: "+"{0:.2f}".format(elapsed)+" s")